In [ ]:
import os
import sys
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.time import Time
import math
import glob
import matplotlib.pyplot as plt
import configparser
import csv
from dotenv import load_dotenv
import logging
import pandas as pd
%matplotlib inline
from astropy import constants as const

In [ ]:
from modules.radial_velocity.src.alg import RadialVelocityAlg
from modules.radial_velocity.src.alg_rv_init import RadialVelocityAlgInit
load_dotenv()
TEST_DIR = os.getenv('KPFPIPE_TEST_DATA') 
print('TEST_DIR:', TEST_DIR)
FIT_G = fitting.LevMarLSQFitter()
LIGHT_SPEED = 299792.458   # light speed in km/s
class DotDict(dict):
    pass
MODULE_DIR = '../../modules/radial_velocity/'

In [ ]:
from tests.regression import test_radial_velocity
print("start test_rv_init_exception")
test_radial_velocity.test_rv_init_exception()
print("test_rv_ccf_init_exception")
test_radial_velocity.test_rv_ccf_init_exception()
#test_radial_velocity.test_compute_rv_by_cc_exception()
#test_radial_velocity.test_neid_compute_rv_by_cc()

In [ ]:
MJD_TO_JD = 2400000.5
class RadialVelocityStats:
    """ This module defines class ' RadialVelocityStats' and methods to do statistic analysis on radial velocity
    results. (this is currently for radial velocity development and testing only).

    Attributes:
         rv_result_set (list): A container storing radial velocity result from fits of level 1 data.
         total_set (int): Total elements in `rv_result_set`.
    """

    def __init__(self, obs_rv_results: list = None):
        self.rv_result_set = list() if obs_rv_results is None else obs_rv_results.copy()
        self.total_set = 0 if obs_rv_results is None else len(obs_rv_results)

    def get_collection(self):
        return self.rv_result_set, self.total_set

    def add_data(self, ccf_rv: float, obj_jd: float):
        self.rv_result_set.append({'jd': obj_jd, 'mean_rv': ccf_rv})
        self.total_set = len(self.rv_result_set)
        return self.rv_result_set, self.total_set

    def analyze_multiple_ccfs(self, ref_date=None):
        """ Statistic analysis on radial velocity numbers of multiple observation resulted by `RadialVelocityAlg`.

        Args:
            ref_date (str, optional): Reference time in the form Julian date format.  Defaults to None.

        Returns:
            dict: Analysis data.

        """
        obs_rvs, total_obs = self.get_collection()
        jd_list = np.array([obs_rv['jd'] for obs_rv in obs_rvs])
        if ref_date is None:
            ref_jd = self.get_start_day(jd_list)
        else:
            ref_jd = Time(ref_date, format='isot', scale='utc').jd
        rv_stats = dict()
        rv_stats['start_jd'] = ref_jd
        rv_stats['hour'] = (jd_list-ref_jd) * 24.0
        rv_stats['day'] = (jd_list-ref_jd)
        rv_stats['values'] = np.array([obs_rv['mean_rv'] for obs_rv in obs_rvs])
        rv_stats['mean'] = np.mean(rv_stats['values'])
        rv_stats['sigma'] = np.std(rv_stats['values'] - rv_stats['mean'])

        return rv_stats

    @staticmethod
    def get_start_day(jd_list: np.ndarray):
        min_jd = np.amin(jd_list)
        day_part = math.floor(min_jd - MJD_TO_JD)
        return MJD_TO_JD+day_part


In [ ]:
# copy of start_logger from logger.py
def get_level(lvl:str) -> int:
    if lvl == 'debug': return logging.DEBUG
    elif lvl == 'info': return logging.INFO
    elif lvl == 'warning': return logging.WARNING
    elif lvl == 'error': return logging.ERROR
    elif lvl == 'critical': return logging.CRITICAL
    else: return logging.NOTSET

def start_logger(logger_name: str, config: str):
    if config is None: 
        # a config file is not provided, so don't start logger
        print('[{}] missing log configuration...not starting a new logger'.format(
            logger_name))
        return None
    config_obj = configparser.ConfigParser()
    res = config_obj.read(config)
    if res == []:
        return None

    log_cfg = config_obj['LOGGER']

    log_start = log_cfg.get('start_log', False)
    log_path = log_cfg.get('log_path', 'log')
    log_lvl = log_cfg.get('log_level', logging.WARNING)
    log_verbose = log_cfg.getboolean('log_verbose', True)
    # logger.setLevel(get_level(log_lvl))
        
    # if log_start:
    #     # setup a log format
    #     formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    #     # setup a log file
    #     f_handle = logging.FileHandler(log_path, mode='w') # logging to file
    #     f_handle.setLevel(get_level(log_lvl))
    #     f_handle.setFormatter(formatter)
    #     logger.addHandler(f_handle)

    #     if log_verbose: 
    #         # also print to terminal 
    #         s_handle = logging.StreamHandler()
    #         s_handle.setLevel(get_level(log_lvl))
    #         s_handle.setFormatter(formatter)
    #         logger.addHandler(s_handle)
    # return logger


    logger = logging.getLogger(logger_name)
    logger.setLevel(get_level(log_lvl))
    logger.propagate = False

    formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    s_handle = logging.StreamHandler()
    s_handle.setLevel(get_level(log_lvl))
    s_handle.setFormatter(formatter)
    logger.addHandler(s_handle)
    return logger


In [ ]:
def plot_gaussian_on_curve(g_curve, curve_x, curve_y, order=None, title=None, ref_curve=None, ref_gaussian=None):
    plt.figure(figsize=(10,10))
    
    label_curve = "rv on order " + str(order) if order is not None else 'all orders '
    plt.plot(curve_x, curve_y, 'ko', label=label_curve)
    if ref_curve is not None:
        plt.plot(curve_x, ref_curve, 'mo', label="neid_ccf"  )
    ref_label = '/'+str("{0:.6f}".format(ref_gaussian.mean.value)) if ref_gaussian is not None else ''
    plt.plot(curve_x, g_curve(curve_x), label='mean:'+str("{0:.6f}".format(g_curve.mean.value))+ ref_label)
                                                      
    plt.legend(loc="lower right", prop={'size': 12})
    plt.title(title) if title is not None else None
    plt.show()

In [ ]:
def write_ccf_to_csv(ccf, csvfile):
    with open(csvfile, mode='w') as result_file:
        result_writer = csv.writer(result_file)
        for i in range(0, np.shape(ccf)[0]):
            row_data = list()
            row_data.append(i)
            for val in ccf[i, :]:
                row_data.append(val)
            result_writer.writerow(row_data)

In [ ]:
def compare_two_csv(file1, file2):
    file1_rows = list()
    file2_rows = list()
    
    with open(file1) as csv1:
        csvReader1 = csv.reader(csv1)
        for row in csvReader1:
            file1_rows.append(row)
    with open(file2) as csv2:
        csvReader2 = csv.reader(csv2)
        for row in csvReader2:
            file2_rows.append(row)
   
    f1 = np.array(file1_rows, dtype=float)
    f2 = np.array(file2_rows, dtype=float)
    
    diff_index = np.where((f1-f2) != 0.0)    
    
    diff_at = list(zip(list(diff_index[0]), list(diff_index[1])))

    return diff_at

In [ ]:
import math
dev = 'developed'
ref = 'reference'

def plot_velocity_time(rv_info, title, label, color, time_unit='hrs', savefig=None):
 
    total_rv = np.size(rv_info[dev]['values'])

    if time_unit == 'hrs':
        rv_delta_time = rv_info[dev]['hour'] 
    else:
        rv_delta_time = rv_info[dev]['day']
        
    plt.figure(figsize=(10,12))     
    s = 100
    ymax = -10000
    ymin = 10000
    for k in rv_info.keys():
        rv_offset = (rv_info[k]['values'] - rv_info[k]['mean']) * 1000.0
        ymax = max(np.amax(rv_offset), ymax)
        ymin = min(np.amin(rv_offset), ymin)
        rv_sigma = rv_info[k]['sigma']*1000.0
        plt.scatter(rv_delta_time, rv_offset, s, c=color[k], edgecolors='b', 
                    label=label[k] + ' sigma = '+ "{:0.6f}".format(rv_sigma)+' m/s')
        

    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times ['+time_unit+']')
    plt.ylabel('RV [m/s]')

    ymax = math.ceil(ymax)+1
    ymin = math.floor(ymin)-1
    xmin = math.ceil(np.amin(rv_delta_time))
    xmax = math.floor(np.amax(rv_delta_time))
    if time_unit == 'hrs':
        plt.xlim((xmin-1, xmax+1))
    else:
        plt.xlim((xmin-20, xmax+20))
    if title is not None:
        plt.title(title)
    
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()    
    

In [ ]:
def make_fits(ccf, mean, out_fits):
    hdu = fits.PrimaryHDU(ccf)
    hdu.header['Date'] = str(datetime.datetime.now())
    if mean is not None: 
        hdu.header['CCF-RVC'] = (str(mean), ' Baryc RV (km/s)')
    hdu.writeto(out_fits, overwrite=True)

In [ ]:
def result_test(target_data, data_result):
    """Check if 2D data is consistent with that from a reference fits.

    Args:
        target_data (numpy.ndarray): Array of data to compare to.
        data_result (numpy.ndarray): Array of data to be checked.

    Returns:

        dict:  Comparison result between the data and the reference data, like::

            {
                'result': 'ok'              # if the data is consistent with the reference.
            }
            {
                'result': 'error',          # if the data is not the same as the reference.
                'msg': <reason message>
            }
    """
    t_y, t_x = np.shape(target_data)
    r_y, r_x = np.shape(data_result)

    if t_y != r_y or t_x != r_x:
        return {'result': 'error', 'msg': 'dimension is not the same'}

    not_nan_data_idx = np.argwhere(~np.isnan(data_result))
    not_nan_target_idx = np.argwhere(~np.isnan(target_data))

    if np.size(not_nan_data_idx) != np.size(not_nan_target_idx):
        return {'result': 'error', 'msg': 'NaN data different'}
    elif np.size(not_nan_data_idx) != 0:
        if not (np.array_equal(not_nan_data_idx, not_nan_target_idx)):
            return {'result': 'error', 'msg': 'NaN data different'}
        else:
            not_nan_target = target_data[~np.isnan(target_data)]
            not_nan_data = data_result[~np.isnan(data_result)]
            diff_idx = np.where(not_nan_target - not_nan_data)[0]

            if diff_idx.size > 0:
                diff_val = not_nan_target - not_nan_data
                diff_max = np.amax(diff_val[diff_idx])
                diff_max_rv = np.amax(data_result[r_y-1, :] - target_data[t_y - 1, :])
                return {'result': 'error', 'msg': 'data is not the same at ' + str(diff_idx.size) +
                                                    ' points and max difference of last row ' + str(diff_max_rv)}

    return {'result': 'ok'}


## RV computation on NEID L1 data and results from Optimal Extraction module

In [ ]:
neid_L1_dir = TEST_DIR + '/NEIDdata/TAUCETI_20191217/L1/neidL1_20191217T'
neid_L2_dir = TEST_DIR + '/NEIDdata/TAUCETI_20191217/L2/neidL2_20191217T'

width_type = 'for_width_3'            # 'for_width_3', 'for_fixed_width', 'for_width_2'
method = 'normal'      #'norect','vertical' or 'normal'

# spectrum_L1_path = '../../modules/optimal_extraction/results/NEID_3sigma/' + width_type + '/NEID_*_extraction_'+method
spectrum_L1_path = '../../test_results/neid/stacked_2fiber_flat_L0_neidTemp_2D20191217T'
outfolder = MODULE_DIR+'results/NEID/'+width_type

s_order=3
e_order= 82
# s_order = 20
# e_order = 24
order_diff = 7
s_x_pos = 600

# import pdb;pdb.set_trace()
spectraname_op = glob.glob(spectrum_L1_path + '*_' + method +'_L1.fits')
spectraname_neid = glob.glob(neid_L1_dir + '*.fits')
spectraname_op.sort()

print('<test>: \n ', spectraname_op)
total_file = len(spectraname_op)

# code_list = [ f[f.index('NEID_0')+len('NEID_'):f.index('_extraction_optimal')] for f in spectraname_op]
d_stamp = '2D20191217T'
code_list = [f[f.index(d_stamp) + len(d_stamp):(f.index(method)-1)] for f in spectraname_op]
print('<test>: \n', 'total ', (len(code_list)), ' code in fits: ', code_list)

config_file = MODULE_DIR + 'configs/default.cfg'
config = configparser.ConfigParser()
config.read(config_file)

logger = start_logger("OrderTraceAlg", config_file)

bc_time = 2458591.5 # 2458278.5 # 2458591.5   # None
corr_path = '../../modules/barycentric_correction/results/'
rv_init = RadialVelocityAlgInit(config, logger, bc_time=bc_time, bc_corr_path=corr_path)
init_result = rv_init.start(print_debug='')


rv_dev_info = RadialVelocityStats()
rv_ref_info = RadialVelocityStats()

In [ ]:
os.getenv('KPFPIPE_TEST_DATA') 

## Prepare ratio table and reweighting result for unit test

In [ ]:
# make ratio table (from NEID L2)

neid_L2_files = sorted(glob.glob(neid_L2_dir+'*.fits'))
reweighting_method = 'ccf_max'
allcpp = []
for f in range(len(neid_L2_files)):
    hdul = fits.open(neid_L2_files[f])
    ccf = hdul[12].data           # a Table HDU
    allcpp.append(np.nanpercentile(ccf[s_order+order_diff:e_order+order_diff+1, :], 95, axis=1))

index = np.where(allcpp == np.max(allcpp))
index = np.squeeze(index[0])
print('index: ', index, neid_L2_files[index])
template_file_hdulist = fits.open(neid_L2_files[index])
template_ccf = (template_file_hdulist[12].data)[(s_order+order_diff):(e_order+order_diff+1), :]

code_str = neid_L2_files[index][len(neid_L2_dir): ]
pos = code_str.find('.fits')
code_str = code_str[0:pos]
print('code: ', code_str)

output_csv = MODULE_DIR+'results/NEID/ccf_ratio_'+ code_str+'_'+str(s_order+order_diff) +'_'+str(e_order+order_diff)+'.csv'
print(output_csv)
rw_ratio_df = RadialVelocityAlg.make_reweighting_ratio_table(template_ccf, s_order+order_diff, 
                                    e_order+order_diff, reweighting_method, max_ratio = 1.0, 
                                    output_csv=output_csv)

In [ ]:
# do reweighting (from NEID L2)

ratio_df = pd.read_csv(output_csv)
total_order = e_order-s_order+1

rv_lev2_info = RadialVelocityStats()
rv_lev2_reweight_info = RadialVelocityStats()
velocities = init_result['data']['velocity_loop'] + 4.0

for f in neid_L2_files:
    print(f)
    neid_L2_file_hdulist = fits.open(f)
    rv_est = neid_L2_file_hdulist[0].header['QRV']
    L1_time = neid_L2_file_hdulist[12].header['CCFJDSUM']
    rv_guess = neid_L2_file_hdulist[12].header['CCFRVSUM']
    ny, nx = np.shape(neid_L2_file_hdulist[12].data)
    rv_lev2_info.add_data(rv_guess, L1_time)

    neid_L2_ccf = neid_L2_file_hdulist[12].data[s_order+order_diff: , :]
    old_L2_ccf = np.zeros((total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS, nx))
    old_L2_ccf[0:total_order, :] = neid_L2_ccf[0:total_order, :]
    old_L2_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :] = velocities
    old_L2_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :] = np.sum(old_L2_ccf[1:total_order, :], axis=0)
    before_ccf_fit, before_ccf_mean, before_x, before_y = RadialVelocityAlg.fit_ccf(old_L2_ccf[-1, :], rv_est,
                                                                                    velocities)
    reweighted_lev2_ccf = RadialVelocityAlg.reweight_ccf(neid_L2_ccf, total_order, ratio_df.values,
                                                         reweighting_method, s_order = s_order+order_diff, 
                                                         do_analysis=True)
    
    rw_lev2_ccf_fit, rw_lev2_ccf_mean, n_x, n_y = RadialVelocityAlg.fit_ccf(reweighted_lev2_ccf[-1, :], rv_est, 
                                                                            velocities)
    make_fits(reweighted_lev2_ccf, rw_lev2_ccf_mean, MODULE_DIR+'results/NEID/reweighted_ccf_'+str(s_order+order_diff) +'_'+str(e_order+order_diff)+'.fits')
    rv_lev2_reweight_info.add_data(rw_lev2_ccf_mean, L1_time) 
    plot_gaussian_on_curve(rw_lev2_ccf_fit, n_x, n_y, ref_curve=before_y, ref_gaussian=before_ccf_fit)
    break

"""
rv_before_stats = rv_lev2_info.analyze_multiple_ccfs()
rv_after_stats = rv_lev2_reweight_info.analyze_multiple_ccfs()
rv_stats = {ref: rv_before_stats, dev: rv_after_stats}
plot_velocity_time(rv_stats, method, {dev: 'after reweighted', ref: 'before reweighted'}, {dev: 'cyan', ref: 'yellow'})
"""

## Radial Velocity computation with or without reweighting ccf orders

In [ ]:
# get reweighting ratio table
out_csv = TEST_DIR + '/radial_velocity_test/results/neid_tauceti_ratio_' + str(s_order) + '_' + str(e_order) + '.csv'
ratio_df = pd.read_csv(out_csv)
ratio_ref = ratio_df.values

for f in range(total_file):
#for f in range(1):
    code = code_list[f]
    spec_name = spectraname_op[f]
    print('File ', f+1, 'of ', total_file, ' file: ', spec_name, ' code: ', code)
    
    L1_data = fits.open(spec_name)
    spec_data = L1_data[1].data
    neid_hdulist = fits.open(neid_L1_dir+code+'.fits')
    print('<test> :','neid L1 file: ', neid_L1_dir+code)

    neid_ref, ref_header = fits.getdata(neid_L2_dir+code+'.fits', header=True)   # ref is not used in this test
    
    order_diff = 7   # order difference between the result from optimal extaction module and NEID L1 data
    h = neid_hdulist[0].header
    wave = neid_hdulist[7].data[order_diff:, :]  # wavelength calibration 
    
    if 'data' not in init_result:
        continue
    radial_vel = RadialVelocityAlg(spec_data, h, init_result, wave,  config, logger)
    
    ny, nx = np.shape(spec_data)
    e_x_pos = nx - s_x_pos
    
    rv_guess = h['QRV'] if 'QRV' in h else init_result['data'][RadialVelocityAlgInit.RV_CONFIG][RadialVelocityAlgInit.START_RV]
    cindy_rv  = radial_vel.compute_rv_by_cc(start_order = s_order, 
                                            end_order = e_order, 
                                            start_x = s_x_pos, 
                                            end_x = e_x_pos, print_progress='')
                                            #  end_x = e_x_pos, ref_ccf=ratio_ref, print_progress='')
    if not cindy_rv:
        continue
    cindy_ccf = cindy_rv['ccf_ary']
    
    total_order = radial_vel.spectrum_order
    gaussian_fit, gaussian_mean, g_x, g_y = radial_vel.fit_ccf(
                                    cindy_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess, 
                                    cindy_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])
                                        
    """ gaussian on reweighted ccf orders
    ref_fits = outfolder+'/rv_optimal_'+method+'_'+code+'.fits'
    ref_hdulist = fits.open(ref_fits)
    ref_data = ref_hdulist[0].data
    ref_fit, ref_mean, ref_x, ref_y = radial_vel.fit_ccf(
                                    ref_data[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess, 
                                    ref_data[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=ref_fits, ref_curve=ref_y, ref_gaussian=ref_fit)
    """
    
    wave = neid_hdulist[7].data
     
    radial_vel_neid = RadialVelocityAlg(neid_hdulist[1].data,  h, init_result, wave, config, logger)  
    # import pdb;pdb.set_trace()
    neid_rv = radial_vel_neid.compute_rv_by_cc(start_order = s_order+order_diff, 
                                                end_order = e_order+order_diff, 
                                                start_x = s_x_pos,
                                                end_x = e_x_pos,  print_progress='')

    neid_ccf = neid_rv['ccf_ary']  
    neid_fit, neid_mean, n_x, n_y = radial_vel.fit_ccf(
                                    neid_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess, 
                                    neid_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])
    
    # outfile_neid = outfolder + '/rv_neid_'+code+'.fits'
    # outfile_cindy = outfolder + '/rv_cindy_'+width_type+'_'+code+'.fits'
       
    L1_time = cindy_rv['jd'] 
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=spectraname_op[f], ref_curve=n_y, ref_gaussian=neid_fit)
    
    # target_file_to_test is for output (make_fits) or comparison (reslt_test)
    # target_file_to_test = '../results/NEID/'+width_type+'/rv_output/'+ 'rv_cindy_'+method+ '_'+code+'.fits'
    # target_data = fits.getdata(target_file_to_test)
    # target_file_to_test = '../results/NEID/'+width_type+'/rv_output/'+ 'rv_'+method+ '_' + code +  \
    #            '_' + str(s_order) + '_' + str(e_order) + '.fits'
    # make_fits(cindy_ccf, gaussian_mean, outfolder+'/rv_optimal_'+method+'_'+code+'.fits')
    
    """
     # for norect only
    target_file_to_test =  '/Users/cwang/documents/KPF/KPF-Pipeline/test_results/neid/tmp/'+ \
             'stacked_2fiber_flat_L0_neidTemp_2D20191217T' + code_list[f] + '_norect_L1_L2.fits'
    
    print('target_file: ', target_file_to_test)
    target_hdu = fits.open(target_file_to_test)
    df = pd.DataFrame(target_hdu[6].data)
    target_data = df.values
    
    compare_result = result_test(target_data, cindy_ccf)
    print(compare_result)
    """    
    rv_dev_results, total_dev_rv = rv_dev_info.add_data(gaussian_fit.mean.value, L1_time)
    rv_ref_results, total_ref_rv = rv_ref_info.add_data(neid_fit.mean.value, L1_time)        

In [ ]:
# find the template observation from above NEID ccf results
lev2_result_files = sorted(glob.glob(outfolder+'/rv_optimal_'+method+'*.fits'))
reweighting_method = 'ccf_max'
allcpp = []
total_order = e_order-s_order+1
for f in range(len(lev2_result_files)):
    hdul = fits.open(lev2_result_files[f])
    ccf = hdul[0].data           # a Table HDU
    allcpp.append(np.max([np.nanpercentile(ccf[od], 95) for od in range(total_order)]))

import pdb;pdb.set_trace()
reweighting_method = 'ccf_max'
index = np.where(allcpp == np.max(allcpp))
index = np.squeeze(index[0])
print(index, lev2_result_files[index])
import pdb;pdb.set_trace()
template_file_hdulist = fits.open(lev2_result_files[index])
template_ccf = template_file_hdulist[0].data
out_csv = MODULE_DIR+'results/NEID/neid_ratio_'+str(s_order)+ '_'+str(e_order)+'.csv'
import pdb;pdb.set_trace()
rw_ratio_df = RadialVelocityAlg.make_reweighting_ratio_table(template_ccf[0:total_order, :], 
                                            s_order, e_order, reweighting_method, max_ratio=1, output_csv=out_csv)


## RV vs. time from the results of above RV computation

In [ ]:
rv_dev_stats = rv_dev_info.analyze_multiple_ccfs()
rv_ref_stats = rv_ref_info.analyze_multiple_ccfs()
rv_stats = {'developed': rv_dev_stats, 'reference': rv_ref_stats}

In [ ]:
print(rv_stats)
plot_velocity_time(rv_stats, method, {dev: 'developed', ref: 'neid'},
                   {dev: 'cyan', ref: 'yellow'})

## RV computation on HARPS L1 data 

In [ ]:
harps_L1_dir = TEST_DIR + '/rv_test/HARPStauceti_baseline/e2ds/'
harps_L2_dir = TEST_DIR + '/rv_test/HARPStauceti_baseline/ccf/'
outfolder = MODULE_DIR+'results/HARPS/'

s_order= 0
e_order= 69
s_x_pos = 500
e_x_pos = 3500

spectraname_harps = glob.glob(harps_L1_dir + 'HARPS.*_e2ds_A.fits')
level2_harps = glob.glob(harps_L2_dir + 'HARPS.*_ccf_G2_A.fits')
spectraname_harps.sort()

#print('<test>: \n ', '\n'.join(spectraname_harps))

k_harps = 'HARPS.20'
k_e2ds = '_e2ds_A'
k_ccf = '_ccf_G2_A'

start_idx = spectraname_harps[0].index(k_harps)+len(k_harps)
end_idx = spectraname_harps[0].index(k_e2ds)
total_file = len(spectraname_harps)

code_list = [ f[start_idx:end_idx] for f in spectraname_harps]
print('<test>: \n', 'total ', (len(code_list)), ' code in fits: ', code_list)

config_file = MODULE_DIR+'configs/default_harps.cfg'
config = configparser.ConfigParser()
config.read(config_file)

logger = start_logger("OrderTraceAlg", config_file)

rv_init = RadialVelocityAlgInit(config, logger)
init_result = rv_init.start(print_debug='')

rv_dev_info = RadialVelocityStats()
rv_ref_info = RadialVelocityStats()
rv_arp_info = RadialVelocityStats()

In [ ]:
rv_guess = init_result['data']['rv_config']['start_rv']
for f in range(total_file):
    code = code_list[f]
    spec_name = spectraname_harps[f]
    print('File ', f+1, 'of ', total_file, ' file: ', spec_name, ' code: ', code)
    spec_data, spec_header = fits.getdata(spec_name, header=True)
    
    publicfile = harps_L2_dir+k_harps+code+k_ccf+'.fits'
    print('<test> :','harps L2 file: ', publicfile )
    harps_ccf, ccf_head = fits.getdata(publicfile, header=True)
    
    radial_vel = RadialVelocityAlg(spec_data, spec_header, init_result, None, config=config, logger=None)
    ref_ccf = harps_ccf[s_order:e_order+1, :]
    cindy_rv  = radial_vel.compute_rv_by_cc(start_order = s_order, 
                                            end_order = e_order, 
                                            start_x = s_x_pos, 
                                            end_x = e_x_pos, ref_ccf = ref_ccf, print_progress="")
    
    cindy_ccf = cindy_rv['ccf_ary']
    total_order = radial_vel.spectrum_order
    gaussian_fit, gaussian_mean, g_x, g_y = radial_vel.fit_ccf(
                            cindy_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess, 
                            cindy_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])         
    new_harps_ccf = harps_ccf.copy()
    new_harps_ccf[71, :] = cindy_ccf[71, :]      # copy the velocity step
    harps_fit, harps_mean, h_x, h_y = radial_vel.fit_ccf(
        new_harps_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess,
        new_harps_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])
    
    harps_mean_head = ccf_head['HIERARCH ESO DRS CCF RVC']
    L1_time = cindy_rv['jd'] 
    print("harps mean: ", harps_mean, harps_mean_head, 'jd: ', L1_time, ccf_head['MJD-OBS'])
       
                   
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=spectraname_harps[f],
                           ref_curve=h_y, ref_gaussian=harps_fit)
    
    # compare to the result before porting to make sure the porting produces the same result
    """
    target_file_to_test = '/Users/cwang/documents/KPF/KPF-Pipeline/AlgorithmDev_07122020/test_data_02242020/rv_test'+\
          '/output_cindy_08042020/HARPStaucetiCindy.20'+code+'_norm_ccf_G2_A.fits'
          
    # target_file_to_test = outfolder+'HARPStaucetiCindy.20'+code+'_norm_ccf_G2_A.fits'   
    # import pdb;pdb.set_trace()    
    print('taget file: ', target_file_to_test)  
    target_data = fits.getdata(target_file_to_test)
    compare_result = result_test(target_data, cindy_ccf)
    print(compare_result)
    """
    
    rv_arpita_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/AlgorithmDev_07122020/test_data_02242020/rv_test/'+\
    '/output_arpita/HARPStaucetiARPITA.20'+code+'_norm_ccf_G2_A.fits'
    arpita_ccf, arpita_head = fits.getdata(rv_arpita_fits, header=True)
    arpita_fit, arpita_mean, a_x, a_y = radial_vel.fit_ccf(
        arpita_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-1, :], rv_guess,
        arpita_ccf[total_order+RadialVelocityAlg.ROWS_FOR_ANALYSIS-2, :])
    
    # make_fits(cindy_ccf, gaussian_mean, target_file_to_test)
        
    rv_dev_results, total_dev_rv = rv_dev_info.add_data(gaussian_fit.mean.value, L1_time)
    rv_ref_results, total_ref_rv = rv_ref_info.add_data(harps_mean_head, L1_time)    
    rv_arp_results, total_arp_rv = rv_arp_info.add_data(arpita_mean, L1_time)

In [ ]:
# comparing data from the module, harps and Arpita's implementation
rv_dev_stats = rv_dev_info.analyze_multiple_ccfs()
rv_ref_stats = rv_ref_info.analyze_multiple_ccfs()
rv_arp_stats = rv_arp_info.analyze_multiple_ccfs()

rv_stats = {'dev2': rv_arp_stats, dev: rv_dev_stats, ref: rv_ref_stats}

plot_velocity_time(rv_stats, "RV for harps", {dev: 'developed', ref: 'harps', 'dev2': 'Arpita'},
                   {dev: 'cyan', ref: 'yellow', 'dev2': 'green'}, time_unit='day')